# Train a SCCNN

In this notebook, we will create and train a High Skip Network in the simplicial complex domain, as proposed in the paper by [Yang et. al : Convolutional Learning on Simplicial Complexes (2023)](https://arxiv.org/abs/2301.11163). 

We train the model to perform binary node classification using the KarateClub benchmark dataset. 

In [122]:
import torch
import numpy as np

from toponetx import SimplicialComplex
import toponetx.datasets.graph as graph

from topomodelx.nn.simplicial.sccnn_layer import SCCNNLayer

# Pre-processing

## Import dataset ##

The first step is to import the Karate Club (https://www.jstor.org/stable/3629752) dataset. This is a singular graph with 34 nodes that belong to two different social groups. We will use these groups for the task of node-level binary classification.

We must first lift our graph dataset into the simplicial complex domain.

In [123]:
dataset = graph.karate_club(complex_type="simplicial")
print(dataset)
max_rank = dataset.dim
print(max_rank)

Simplicial Complex with shape (34, 78, 45, 11, 2) and dimension 4
4


In [124]:
incidence_1 = dataset.incidence_matrix(rank=1)
incidence_2 = dataset.incidence_matrix(rank=2)

print(f"The incidence matrix B1 has shape: {incidence_1.shape}.")
print(f"The incidence matrix B2 has shape: {incidence_2.shape}.")

The incidence matrix B1 has shape: (34, 78).
The incidence matrix B2 has shape: (78, 45).


In [125]:
laplacian_0  = dataset.hodge_laplacian_matrix(rank=0,weight=True)
laplacian_down_1 = dataset.down_laplacian_matrix(rank=1,weight=True)
laplacian_up_1 = dataset.up_laplacian_matrix(rank=1,weight=True)
laplacian_down_2 = dataset.down_laplacian_matrix(rank=2,weight=True)
laplacian_up_2 = dataset.up_laplacian_matrix(rank=2,weight=True)

laplacian_0  = dataset.adjacency_matrix(rank=0,weight=True)
laplacian_down_1 = dataset.coadjacency_matrix(rank=1,weight=True)
laplacian_up_1 = dataset.adjacency_matrix(rank=1,weight=True)
laplacian_down_2 = dataset.coadjacency_matrix(rank=2,weight=True)
laplacian_up_2 = dataset.adjacency_matrix(rank=2,weight=True)

In [126]:

import scipy.sparse as sp
import scipy.sparse.linalg as spl

# def normalize(L, half_interval = False):
#     assert(sp.isspmatrix(L))
#     M = L.shape[0]
#     assert(M == L.shape[1])
#     topeig = spl.eigsh(L, k=1, which="LM", return_eigenvectors = False)[0]   
#     #print("Topeig = %f" %(topeig))

#     ret = L.copy()
#     if half_interval:
#         ret *= 1.0/topeig
#     else:
#         ret *= 2.0/topeig
#         ret.setdiag(ret.diagonal(0) - np.ones(M), 0)

#     return ret

# laplacian_0 = normalize(laplacian_0)
# laplacian_down_1 = normalize(laplacian_down_1)
# laplacian_up_1 = normalize(laplacian_up_1)
# laplacian_down_2 = normalize(laplacian_down_2)
# laplacian_up_2 = normalize(laplacian_up_2)

# def normalize_incidence(B):
#     row_sums = B.sum(axis=1)
#     B = B/row_sums[:,np.newaxis]

# def normalizeRows(x):
#     """
#     Implement a function that normalizes each row of the matrix x (to have unit length).
    
#     Argument:
#     x -- A numpy matrix of shape (n, m)
    
#     Returns:
#     x -- The normalized (by row) numpy matrix. You are allowed to modify x.
#     """
    
#     # Compute x_norm as the norm 2 of x. Use np.linalg.norm(..., ord = 2, axis = ..., keepdims = True)
#     x_norm = np.linalg.norm(x, axis=1, keepdims=True)
#     print("x_norm.shape:", x_norm.shape, "\n")
    
#     # Divide x by its norm.
#     x = x / x_norm

#     return x    

# incidence_1 = sp.csr_matrix(normalizeRows(incidence_1.toarray()))
# incidence_2 = sp.csr_matrix(normalizeRows(incidence_2.toarray()))
# print(incidence_1)


# # from scipy.sparse.linalg import eigsh
# # from scipy import sparse 

# # eig_0, _ = eigsh(laplacian_0,k=1, which='LM')
# # laplacian_0 = sparse.csr_matrix(laplacian_0/eig_0)
# # eig_down_1, _ = eigsh(laplacian_down_1,k=1,which='LM')
# # laplacian_down_1 = sparse.csr_matrix(laplacian_down_1/eig_down_1)
# # eig_up_1, _ = eigsh(laplacian_up_1,k=1,which='LM')
# # laplacian_up_1 = sparse.csr_matrix(laplacian_up_1/eig_up_1)
# # eig_down_2, _ = eigsh(laplacian_down_2,k=1,which='LM')
# # laplacian_down_2 = sparse.csr_matrix(laplacian_down_2/eig_down_2)
# # eig_up_2, _ = eigsh(laplacian_up_2,k=1,which='LM')
# # laplacian_up_2 = sparse.csr_matrix(laplacian_up_2/eig_up_2)


In [127]:

laplacian_0 = torch.from_numpy(laplacian_0.todense()).to_sparse()
laplacian_down_1 = torch.from_numpy(laplacian_down_1.todense()).to_sparse()
laplacian_up_1 = torch.from_numpy(laplacian_up_1.todense()).to_sparse()
laplacian_down_2 = torch.from_numpy(laplacian_down_2.todense()).to_sparse()
laplacian_up_2 = torch.from_numpy(laplacian_up_2.todense()).to_sparse()

incidence_1 = torch.from_numpy(incidence_1.todense()).to_sparse()
incidence_2 = torch.from_numpy(incidence_2.todense()).to_sparse()


## Import signal ##

Since our task will be node classification, we must retrieve an input signal on the nodes. The signal will have shape $n_\text{nodes} \times$ in_channels, where in_channels is the dimension of each cell's feature. Here, we have in_channels = channels_nodes $ = 34$. This is because the Karate dataset encodes the identity of each of the 34 nodes as a one hot encoder.

In [128]:
"""A function to obtain features based on the input: rank
"""
def get_simplicial_features(dataset,rank):
    if rank == 0: 
        which_feat = "node_feat"
    elif rank == 1:
        which_feat = "edge_feat"
    elif rank == 2:
        which_feat = "face_feat"
    else:
        raise ValueError(f"input dimension must be 0, 1 or 2, because features are supported on nodes, edges and faces") 
    
    x = []
    for _, v in dataset.get_simplex_attributes(which_feat).items():
        x.append(v)
    
    x = torch.tensor(np.stack(x))
    return x

## Define binary labels
We retrieve the labels associated to the nodes of each input simplex. In the KarateClub dataset, two social groups emerge. So we assign binary labels to the nodes indicating of which group they are a part.

We convert the binary labels into one-hot encoder form, and keep the first four nodes' true labels for the purpose of testing.

In [129]:
y = np.array(
    [
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        0,
        1,
        1,
        1,
        1,
        0,
        0,
        1,
        1,
        0,
        1,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ]
)
y_true = np.zeros((34, 2))
y_true[:, 0] = y
y_true[:, 1] = 1 - y
y_test = y_true[-4:]
y_train = y_true[:30]

y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

# Create the SCCNN

In [130]:
class SCCNN(torch.nn.Module):
    """SCCNN implementation for binary node classification 
    Note: In this task, we direcly consider the finaly output on the nodes, which is passed by a linear layer, as the label output. 

    Parameters
    """
    def __init__(self, in_channels_all, intermediate_channels_all,out_channels_all, conv_order, sc_order, aggr_norm=True,update_func="sigmoid",n_layers=2):
        super().__init__()
        # first layer 
        layers = [SCCNNLayer(in_channels=in_channels_all,out_channels=intermediate_channels_all,conv_order=conv_order,sc_order=sc_order,aggr_norm=aggr_norm,update_func=update_func)]

        for _ in range(n_layers-1):
            layers.append(
                SCCNNLayer(in_channels=intermediate_channels_all,out_channels=out_channels_all,conv_order=conv_order,sc_order=sc_order,aggr_norm=aggr_norm,update_func=update_func)
            )
        out_channels_0, out_channels_1, out_channels_2 = out_channels_all
        self.linear = torch.nn.Linear(out_channels_0,2)
        self.layers = layers 

    def forward(self,x_all,laplacian_all,incidence_all):
        """Forward computation. 
        
        Parameters
        ----------
        """
        for layer in self.layers:
            x_all = layer(x_all,laplacian_all,incidence_all)
        
        """
        We pass the output on the ndoes to a linear layers and use that for labels
        """
        x_0, _, _ = x_all 
        logits = self.linear(x_0)
        label = torch.sigmoid(logits)
        
        return label

# Train the Neural Network

We specify the model with our pre-made neighborhood structures and specify an optimizer.

In [131]:
"""Obtain the initial features on all simplices"""
x_0 = get_simplicial_features(dataset,rank=0)
x_1 = get_simplicial_features(dataset,rank=1)
x_2 = get_simplicial_features(dataset,rank=2)

x_all = (x_0,x_1,x_2)

conv_order = 5
in_channels_all = (x_0.shape[-1],x_1.shape[-1],x_2.shape[-1])
intermediate_channels_all = (4,4,4)
out_channels_all = intermediate_channels_all
num_layers = 2

laplacian_all = (laplacian_0,laplacian_down_1,laplacian_up_1,laplacian_down_2,laplacian_up_2)

incidence_all = (incidence_1,incidence_2)

model = SCCNN(in_channels_all=in_channels_all,intermediate_channels_all=intermediate_channels_all,out_channels_all=out_channels_all,conv_order=conv_order,sc_order=max_rank,n_layers=num_layers)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


In [132]:
test_interval = 1
num_epochs = 2
for epoch_i in range(1, num_epochs + 1):
    epoch_loss = []
    model.train()
    optimizer.zero_grad()
    y_hat = model(x_all, laplacian_all, incidence_all)
    loss = torch.nn.functional.binary_cross_entropy(
        y_hat[: len(y_train)].float(), y_train.float()
    )
    epoch_loss.append(loss.item())
    loss.backward()
    optimizer.step()

    y_pred = torch.where(y_hat > 0.5, torch.tensor(1), torch.tensor(0))
    accuracy = (y_pred[-len(y_train) :] == y_train).all(dim=1).float().mean().item()
    print(
        f"Epoch: {epoch_i} loss: {np.mean(epoch_loss):.4f} Train_acc: {accuracy:.4f}",
        flush=True,
    )
    if epoch_i % test_interval == 0:
        with torch.no_grad():
            y_hat_test = model(x_all, laplacian_all, incidence_all)
            y_pred_test = torch.where(
                y_hat_test > 0.5, torch.tensor(1), torch.tensor(0)
            )
            test_accuracy = (
                torch.eq(y_pred_test[-len(y_test) :], y_test)
                .all(dim=1)
                .float()
                .mean()
                .item()
            )
            print(f"Test_acc: {test_accuracy:.4f}", flush=True)

Epoch: 1 loss: 0.8414 Train_acc: 0.4333
Test_acc: 1.0000
Epoch: 2 loss: 0.8338 Train_acc: 0.4333
Test_acc: 1.0000
